In [3]:
with open('to_download.txt') as f:
    download_list = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
download_list = [int(x.strip()) for x in download_list]

In [30]:
import shelve
from time import sleep
import vk_api
from vk_api import VkUpload
from vk_api.longpoll import VkLongPoll, VkEventType
import requests
from vk_api.execute import VkFunction
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import json
import datetime
import numpy as np
import shelve


import settings

In [27]:
def isCommunityAlive(comm_id, vk):
    try:
        info = vk.wall.get(owner_id=-comm_id, owners_only=1, count=2)
        last_post_date = max(datetime.datetime.fromtimestamp(info.get('items')[0].get('date')),
                             datetime.datetime.fromtimestamp(info.get('items')[1].get('date')))
        last_count = datetime.datetime.now() - last_post_date
        if (last_count.days < 30):
            return (comm_id, "Alive")
        else:
            return (comm_id, "Dead")
    except  vk_api.exceptions.ApiError as e:
        return (comm_id, e.__str__())

In [28]:
%%time 


vk_session = vk_api.VkApi(login=settings.MY_USER_LOGIN, password=settings.MY_USER_PASSWORD)
try:
    vk_session.auth(reauth=True, token_only=True)
except vk_api.AuthError as error_msg:
    print(error_msg)
    
vk = vk_session.get_api()

ans = [isCommunityAlive(group_id, vk) for group_id in download_list]

CPU times: user 853 ms, sys: 38.7 ms, total: 892 ms
Wall time: 45.9 s


In [31]:
File = shelve.open("data.dat")
File['info'] = ans
File.close()